In [68]:
import pandas as pd
import numpy as np
from src.exp_helper import *
from src.utils.spatial_unit import grid2nbh


In [75]:
grid_size = 50
train_tw = 60
verbose = 0
data = CompileData(spu_name='grid_%d' % grid_size)
data.set_x(['crime'], category_groups={'crime': [['burglary']]}, by_category=False)
data.set_y('crime/burglary')

****WARNING**** Some data get 0 assignment
****WARNING**** Some data get 0 assignment


In [79]:
bower = pd.read_pickle('tmp/bower.pickle').to_dense()
ys = pd.read_pickle('tmp/true_y.pickle').to_dense()
kde200 = pd.read_pickle('tmp/kde.pickle')
xday=2

In [80]:
# def bnia_fpn(data, ys, bower,kde200, xday=2):
top20 = lambda x: x > np.percentile(x, 80)
above_mean = lambda x: x > x.mean()
above_mean_std = lambda x: x > (x.mean() + x.std())
hotspot_def = [('above_mean', above_mean), ('top20', top20), ('above_mean_std', above_mean_std)]

for hname, hfunc in hotspot_def:
    for fname in ['fp','fn']:
        fpn_res = {}
        for p in ys.columns:
            p_fpn_res = {}
            for mname, model in [('bower', bower),('kde200',kde200)]:
                h = hfunc(model[p])  # is hotspot or not
                y = ys[p].astype(bool)  # has crime or not
                fpn = (h&~y) if fname=='fp' else (~h&y)
                fpn = pd.Series(fpn, index=data.spu.index, name='stat')
                fpn = grid2nbh(fpn).values.tolist()
                p_fpn_res['%s' % (mname)] = fpn
            fpn_res[p]=p_fpn_res
        pd.DataFrame.from_dict(fpn_res, 'index').to_csv('tmp/bower_%dday_bnia_%s_hotspots_%s.csv' % (xday, hname, fname))
